In [147]:
import pandas as pd
import matplotlib.pyplot as plt  # 그래프 그리는 라이브러리
from sklearn.model_selection import train_test_split
from sklearn import metrics  # 평가를 위한 라이브러리
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import seaborn as sns  # 시각화 라이브러리

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import VotingClassifier  # 보팅 분류기, 앙상블을 결합시켜주는 역할
from sklearn.ensemble import RandomForestClassifier

In [122]:
train = pd.read_csv('smhrd111/train.csv', index_col = 'ID')
test = pd.read_csv('smhrd111/test.csv', index_col = 'ID')

In [123]:
train.describe()

,stag,event,age,extraversion,independ,selfcontrol,anxiety,novator
count,516.000000,699.000000,699.000000,699.000000,699.000000,666.000000,699.000000,699.000000
mean,36.844086,0.649499,30.470962,5.631187,5.550787,5.554054,5.584263,5.834907
std,34.352824,0.477468,6.736574,1.814112,1.727731,1.934978,1.728112,1.935977
min,0.492813,0.000000,18.000000,1.000000,1.000000,1.000000,1.700000,1.000000
25%,12.082136,0.000000,25.000000,4.600000,4.100000,4.100000,4.000000,4.400000
50%,25.544148,1.000000,29.000000,5.400000,5.500000,5.700000,5.600000,6.000000
75%,49.322382,1.000000,35.000000,7.000000,6.900000,7.200000,7.100000,7.500000
max,179.449692,1.000000,56.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [124]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430 entries, 700 to 1129
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stag          321 non-null    float64
 1   gender        430 non-null    object 
 2   age           430 non-null    float64
 3   industry      430 non-null    object 
 4   profession    430 non-null    object 
 5   traffic       430 non-null    object 
 6   coach         430 non-null    object 
 7   head_gender   360 non-null    object 
 8   greywage      430 non-null    object 
 9   way           430 non-null    object 
 10  extraversion  430 non-null    float64
 11  independ      430 non-null    float64
 12  selfcontrol   385 non-null    float64
 13  anxiety       430 non-null    float64
 14  novator       430 non-null    float64
dtypes: float64(7), object(8)
memory usage: 53.8+ KB


In [125]:
test.corr()

,stag,age,extraversion,independ,selfcontrol,anxiety,novator
stag,1.000000,-0.213404,-0.129267,0.065200,0.050795,-0.040311,-0.066284
age,-0.213404,1.000000,-0.116970,0.043920,0.013688,0.038650,0.062018
extraversion,-0.129267,-0.116970,1.000000,-0.191124,-0.579729,-0.107014,0.317611
independ,0.065200,0.043920,-0.191124,1.000000,-0.121016,-0.436577,0.002716
selfcontrol,0.050795,0.013688,-0.579729,-0.121016,1.000000,-0.137211,-0.526195
anxiety,-0.040311,0.038650,-0.107014,-0.436577,-0.137211,1.000000,0.227025
novator,-0.066284,0.062018,0.317611,0.002716,-0.526195,0.227025,1.000000


In [126]:
train.drop('head_gender', axis = 1 , inplace = True)
test.drop('head_gender', axis = 1 , inplace = True)

In [127]:
train.drop('greywage', axis = 1 , inplace = True)
test.drop('greywage', axis = 1 , inplace = True)

In [128]:
train['way'] = train['way'].fillna('bus')

In [129]:
pd.set_option('display.max_row', 500)
pt1 = train.pivot_table(values = 'stag', index = ['gender', 'way'], aggfunc = 'mean')
pt1

stag
gender way            
f      bus   33.751807
       car   39.259479
       foot  36.116163
m      bus   35.993101
       car   54.145197
       foot  21.942961

In [130]:
def fill_stag(df):
    # 만약 나이가 결측치라면 피봇테이블에서 값을 가져와서 채운다.
    if np.isnan(df['stag']): # 결측치이면 true, 아니면 false
        return pt1.loc[df['gender'],df['way']]
    # 만약 나이가 결측치가 아니라면 원래 나이값을 사용하자
    else:
        return df['stag']

In [131]:
train['stag'] = train.apply(fill_stag, axis = 1).astype('float64')

In [132]:
test['stag'] = test.apply(fill_stag, axis = 1).astype('float64')

In [133]:
train['industry'] = train['industry'].fillna('Retail')

In [134]:
train['way'] = train['way'].fillna('bus')

In [135]:
train['selfcontrol'] = train['selfcontrol'].fillna(5.7)

In [136]:
test['selfcontrol'] = test['selfcontrol'].fillna(6.4)

In [137]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 699 entries, 1 to 699
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stag          699 non-null    float64
 1   event         699 non-null    int64  
 2   gender        699 non-null    object 
 3   age           699 non-null    float64
 4   industry      699 non-null    object 
 5   profession    699 non-null    object 
 6   traffic       699 non-null    object 
 7   coach         699 non-null    object 
 8   way           699 non-null    object 
 9   extraversion  699 non-null    float64
 10  independ      699 non-null    float64
 11  selfcontrol   699 non-null    float64
 12  anxiety       699 non-null    float64
 13  novator       699 non-null    float64
dtypes: float64(7), int64(1), object(6)
memory usage: 81.9+ KB


In [138]:
y_train = train['event']
X_train = train.drop('event', axis = 1)
X_test = test

In [139]:
cat_feature = ['gender', 'industry', 'profession', 'traffic', 'coach', 'way']

In [140]:
for i in cat_feature:
    dummy = pd.get_dummies(train[i], prefix = i)
    X_train = pd.concat([X_train, dummy], axis = 1)
    X_train.drop(i, axis=1, inplace = True)

In [141]:
for i in cat_feature:
    dummy = pd.get_dummies(test[i], prefix = i)
    X_test = pd.concat([X_test, dummy], axis = 1)
    X_test.drop(i, axis=1, inplace = True)

In [142]:
set(X_train.columns) - set(X_test.columns)

set()

In [143]:
tree_model = DecisionTreeClassifier(max_depth = 10)

In [144]:
result = cross_val_score(tree_model,
                        X_train,
                        y_train,
                        cv = 10)
result

array([0.57142857, 0.5       , 0.64285714, 0.74285714, 0.8       ,
       0.57142857, 0.67142857, 0.51428571, 0.65714286, 0.69565217])

In [145]:
result.mean()

0.6367080745341616

### 그리드서치

In [148]:
forest_model2 = RandomForestClassifier(n_estimators = 1000,
                                      max_features = 0.7,
                                      max_depth = 5,
                                      min_samples_leaf = 15,
                                      max_leaf_nodes = 50)

In [149]:
from sklearn.model_selection import GridSearchCV

In [150]:
param_grid = {
    "max_depth" : [5, 10, 15, 20],
    "n_estimators" : [1000, 1500, 2000, 2500],
    "max_features" : [0.5, 0.7, 0.9],
    "max_leaf_nodes" : [20, 50, 80]
}

In [151]:
grid = GridSearchCV(forest_model2,
                   param_grid,
                   cv = 3)

In [ ]:
grid.fit(X_train, y_train)